In [1]:
# 0. [개인 경로만 입력] ------------------------------------------------------
# 아래 변수만 바꿔서 사용하세요! / Only change this variable for your own path!
MY_PATH = "sample_img/action_video"  # 예: "sample_img/sample_video"

# ---------------------------------------------------------------------------

# 1. Google Drive 마운트 / Google Drive Mount (stable)
from google.colab import drive
drive.flush_and_unmount()  # 기존 마운트 해제 / Unmount previous drive
drive.mount('/content/gdrive', force_remount=True)
import os
os.chdir('/content')  # 작업 디렉토리 고정 / Set working directory

# 2. 필수 패키지 버전 체크 및 필요시 설치 / Check and install required packages
import importlib.util
from packaging import version
from importlib.metadata import version as pkg_version

required = {
    'torch': '2.0.1',
    'torchvision': '0.15.2',
    'torchaudio': '2.0.2',
    'mmcv': '2.0.1',
    'mmdet': '<3.3.0',
    'mmpose': '>=1.1.0',
    'mmengine': None
}

install_needed = False

print("🔍 패키지 버전 확인 중... / Checking package versions...")
for pkg, req_ver in required.items():
    try:
        installed_ver = pkg_version(pkg)
        print(f"✅ {pkg}: {installed_ver}")
        if req_ver:
            if req_ver.startswith(('<', '>', '=')):
                req_spec = version.parse(req_ver.replace('=', '').replace('<', '').replace('>', ''))
                if '<' in req_ver and version.parse(installed_ver) >= req_spec:
                    print(f"❌ {pkg}는 {req_ver} 필요, 현재 {installed_ver} / {pkg} requires {req_ver}, but found {installed_ver}")
                    install_needed = True
                elif '>=' in req_ver and version.parse(installed_ver) < req_spec:
                    print(f"❌ {pkg}는 {req_ver} 필요, 현재 {installed_ver} / {pkg} requires {req_ver}, but found {installed_ver}")
                    install_needed = True
            elif version.parse(installed_ver) != version.parse(req_ver):
                print(f"❌ {pkg}는 {req_ver} 필요, 현재 {installed_ver} / {pkg} requires {req_ver}, but found {installed_ver}")
                install_needed = True
    except:
        print(f"❌ {pkg} 미설치 / {pkg} not installed")
        install_needed = True

if install_needed:
    print("\n🚀 필수 패키지 설치 중... / Installing required packages...")
    # 패키지 설치 / Install commands
    !pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118
    !pip uninstall -y jax jaxlib
    !pip install numpy==1.24.4 pandas==1.5.3 scipy==1.10.1 requests==2.28.2 tqdm==4.65.2 filelock==3.14.0
    !pip uninstall -y transformers accelerate torchao
    !pip install transformers==4.30.2
    !pip install -U openmim
    !mim uninstall mmengine mmcv mmdet mmpose -y
    !mim install mmengine
    !pip install mmcv==2.0.1 -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.0/index.html
    !mim install "mmdet<3.3.0"
    !mim install "mmpose>=1.1.0"
    # MMpose 소스 설치 / Install mmpose from source
    %cd /content
    if not os.path.exists('/content/mmpose'):
        !git clone https://github.com/open-mmlab/mmpose.git
    %cd mmpose
    !pip install -r requirements.txt
    !pip install -v -e .
    %cd /content
    print("\n✅ 설치 완료! 런타임을 재시작 후 다시 실행하세요. / Installation completed. Please **RESTART RUNTIME** then run remaining cells!")
else:
    print("\n✅ 모든 패키지가 올바른 버전으로 설치되어 있습니다! / All packages are already installed with correct versions!")

# 3. 작업 폴더 생성 / Create working folders
local_work_root = '/content/work'
local_video_dir = f'{local_work_root}/video'
local_json_dir = f'{local_work_root}/json'
local_vis_dir = f'{local_work_root}/vis'
local_done_vis = f'{local_work_root}/done_vis'
local_done_csv = f'{local_work_root}/done_csv'
local_alljt_vis = f'{local_work_root}/ALL_JT_VIS'
local_alljt_csv = f'{local_work_root}/ALL_JT_CSV'

for d in [local_video_dir, local_json_dir, local_vis_dir, local_done_vis, local_done_csv, local_alljt_vis, local_alljt_csv]:
    os.makedirs(d, exist_ok=True)
!chmod 777 -R /content/work

# 4. 비디오 복사 (Google Drive → Colab 로컬) / Copy videos from Drive to Colab local
import shutil
drive_video_dir = f'/content/gdrive/MyDrive/{MY_PATH}/'
if os.path.exists(local_video_dir):
    shutil.rmtree(local_video_dir)
os.makedirs(local_video_dir, exist_ok=True)
for fname in os.listdir(drive_video_dir):
    if fname.lower().endswith(('.mp4', '.mov')):
        src = os.path.join(drive_video_dir, fname)
        dst = os.path.join(local_video_dir, fname)
        shutil.copy(src, dst)
        print(f"Copied: {fname} (Google Drive → Colab)")

# 5. MMpose 추론 (직접 Inferencer 호출) / MMpose inference (direct Inferencer call)
print("\n🏃‍♀️ MMpose 추론 시작... / Starting MMpose inference...")
from mmpose.apis import MMPoseInferencer

# 모델 및 체크포인트 경로 설정 (이미 다운로드되었거나 캐시되어 있어야 함)
# MMpose 클론 경로를 사용하여 설정 파일을 지정
mmpose_root = '/content/mmpose' # MMpose가 클론된 경로
pose2d_config = f'{mmpose_root}/configs/body_2d_keypoint/topdown_heatmap/coco/td-hm_hrnet-w32_8xb64-210e_coco-256x192.py'
pose2d_checkpoint = 'https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth'

# Inferencer 초기화
try:
    # 수정된 모델 초기화 방식 (직접 URL 대신 모델 alias 사용)
    inferencer = MMPoseInferencer(
        pose2d='td-hm_hrnet-w32_8xb64-210e_coco-256x192',
        device='cpu'  # GPU 가속 사용 / CPU 사용 시 변경
    )
except Exception as e:
    print(f"❌ MMpose Inferencer 초기화 실패: {e}")
    print("MMpose 설치 또는 설정에 문제가 있을 수 있습니다. 런타임 재시작 및 패키지 설치를 다시 확인하세요.")
    inferencer = None # 초기화 실패 시 None으로 설정

video_list = [f for f in os.listdir(local_video_dir) if f.lower().endswith(('.mp4', '.mov'))]

if inferencer:
    for fname in video_list:
        input_path = os.path.join(local_video_dir, fname)
        print(f"처리 중: {fname} / Processing: {fname}")
        try:
            # 추론 실행 및 결과 저장
            # generator를 통해 결과를 얻고, 시각화 및 예측 결과를 저장합니다.
            result_generator = inferencer(input_path,
                                          radius=15,
                                          thickness=8,
                                          pred_out_dir=local_json_dir,
                                          vis_out_dir=local_vis_dir)

            # 결과를 실제로 얻어오기 위해 순회합니다.
            # 이 과정에서 시각화된 비디오 파일과 JSON 파일이 생성됩니다.
            for _ in result_generator:
                pass # generator를 소모하여 파일 생성을 완료합니다.

            print(f"✅ 처리 완료: {fname} / Processed: {fname}")

        except Exception as e:
            print(f"❌ 추론 중 에러 발생 ({fname}): {e} / Error during inference ({fname}): {e}")
            print("해당 비디오 파일 또는 MMpose 설정에 문제가 있을 수 있습니다.")

print("\n✅ MMpose 추론 완료! / MMpose inference completed!")
# 6. Rt.Shoulder 각도만 자막 오버레이 + CSV (done_vis, done_csv)
import cv2
import json
import math
import pandas as pd
import numpy as np

def getAngle(a, b, c):
    # 관절 각도 계산 함수 / Joint angle calculation function
    jt_ang = abs(math.degrees(math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0])))
    if jt_ang > 180:
        jt_ang = 360 - jt_ang
    return jt_ang

for fname in video_list:
    video_name, ext = os.path.splitext(fname)
    vis_video_path = os.path.join(local_vis_dir, f"{video_name}{ext}")
    json_path = os.path.join(local_json_dir, f"{video_name}.json")
    out_video_path = os.path.join(local_done_vis, f"{video_name}_rt_shld{ext}")
    out_csv_path = os.path.join(local_done_csv, f"{video_name}_rt_shld.csv")
    if not (os.path.exists(vis_video_path) and os.path.exists(json_path)):
        print(f"❌ 파일 없음: {vis_video_path} 또는 {json_path} / Missing file: {vis_video_path} or {json_path}")
        continue

    with open(json_path, 'r') as f:
        json_data = json.load(f)

    cap = cv2.VideoCapture(vis_video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(out_video_path, fourcc, fps, (width, height))

    Rt_SHLD = []
    timestamps = []
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret or frame_count >= len(json_data):
            break
        try:
            kp = json_data[frame_count]['instances'][0]['keypoints']
            right_shoulder = kp[6][:2]
            right_elbow = kp[8][:2]
            right_hip = kp[12][:2]
            angle = getAngle(right_elbow, right_shoulder, right_hip)
        except Exception:
            angle = 0.0

        Rt_SHLD.append(angle)
        timestamps.append(frame_count / fps)
        # Rt.Shoulder 각도만 상단 왼쪽에 숫자만 표시 / Only number, no 'deg'
        cv2.putText(frame, f"Right Shoulder Angle: {angle:.2f}", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2, cv2.LINE_AA)
        cv2.putText(frame, f"Right Shoulder Angle: {angle:.2f}", (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 1, cv2.LINE_AA)
        out.write(frame)
        frame_count += 1

    cap.release()
    out.release()
    pd.DataFrame({'Time (s)': timestamps, 'Right Shoulder Angle': Rt_SHLD}).to_csv(out_csv_path, index=False)
    print(f"✅ 저장됨: {out_video_path}, {out_csv_path} / Saved: {out_video_path}, {out_csv_path}")

# 7. 모든 관절 각도 자막/CSV (상단 여러 줄, 각 관절 위치에 각도값 숫자만)
def put_multiline_subtitle(frame, lines, start_y=40, font_scale=0.8, color=(255,255,255), thickness=2):
    # 여러 줄 자막을 상단에 표시 / Draw multi-line subtitles at the top
    for i, text in enumerate(lines):
        y = start_y + i*35
        cv2.putText(frame, text, (40, y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0,0,0), thickness+2, cv2.LINE_AA)
        cv2.putText(frame, text, (40, y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, thickness, cv2.LINE_AA)

def put_joint_angles(frame, keypoints, joint_angles, angles_dict):
    # 관절 위치에 각도값 숫자만 표시 / Draw only angle number at joint positions
    for i, (name, (a, b, c)) in enumerate(joint_angles):
        x, y = keypoints[b][:2]
        angle = angles_dict[name][-1]
        cv2.circle(frame, (int(x), int(y)), 8, (0, 255, 0), -1)
        text = f"{angle:.1f}"
        cv2.putText(frame, text, (int(x)-20, int(y)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 2, cv2.LINE_AA)
        cv2.putText(frame, text, (int(x)-20, int(y)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 1, cv2.LINE_AA)

joint_angles = [
    ("Right Elbow", [6, 8, 10]),
    ("Left Elbow", [5, 7, 9]),
    ("Right Shoulder", [8, 6, 12]),
    ("Left Shoulder", [7, 5, 11]),
    ("Right Knee", [12, 14, 16]),
    ("Left Knee", [11, 13, 15]),
    ("Right Hip", [6, 12, 14]),
    ("Left Hip", [5, 11, 13]),
    ("Neck", [5, 0, 6]),
    ("Waist(L)", [5, 11, 12]),
    ("Waist(R)", [6, 12, 11]),
]

for fname in video_list:
    video_name, ext = os.path.splitext(fname)
    vis_video_path = os.path.join(local_vis_dir, f"{video_name}{ext}")
    json_path = os.path.join(local_json_dir, f"{video_name}.json")
    out_video_path = os.path.join(local_alljt_vis, f"{video_name}_alljt{ext}")
    out_csv_path = os.path.join(local_alljt_csv, f"{video_name}_alljt.csv")
    if not (os.path.exists(vis_video_path) and os.path.exists(json_path)):
        print(f"❌ 파일 없음: {vis_video_path} 또는 {json_path} / Missing file: {vis_video_path} or {json_path}")
        continue

    with open(json_path, 'r') as f:
        json_data = json.load(f)

    cap = cv2.VideoCapture(vis_video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(out_video_path, fourcc, fps, (width, height))

    angle_dict = {name: [] for name, _ in joint_angles}
    timestamps = []
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret or frame_count >= len(json_data):
            break
        try:
            kp = json_data[frame_count]['instances'][0]['keypoints']
            for name, (a, b, c) in joint_angles:
                angle = getAngle(kp[a][:2], kp[b][:2], kp[c][:2])
                angle_dict[name].append(angle)
            subtitle_lines = []
            for i in range(0, len(joint_angles), 3):
                line = ""
                for j in range(i, min(i+3, len(joint_angles))):
                    name, _ = joint_angles[j]
                    angle = angle_dict[name][-1]
                    line += f"{name}: {angle:.1f}  "
                subtitle_lines.append(line)
            put_multiline_subtitle(frame, subtitle_lines, start_y=40)
            put_joint_angles(frame, kp, joint_angles, angle_dict)
        except Exception as e:
            print(f"Frame {frame_count} error: {e}")
            for name, _ in joint_angles:
                angle_dict[name].append(0.0)
        timestamps.append(frame_count / fps)
        out.write(frame)
        frame_count += 1

    cap.release()
    out.release()
    df = pd.DataFrame({'Time (s)': timestamps})
    for name in angle_dict:
        df[name] = angle_dict[name]
    df.to_csv(out_csv_path, index=False)
    print(f"✅ 저장됨: {out_video_path}, {out_csv_path} / Saved: {out_video_path}, {out_csv_path}")

# 8. 결과를 Google Drive로 복사 / Copy results to Google Drive
def safe_copy(src_pattern, dst_folder):
    import glob
    files = glob.glob(src_pattern)
    if not files:
        print(f"No files to copy: {src_pattern}")
        return
    os.makedirs(dst_folder, exist_ok=True)
    for file in files:
        try:
            shutil.copy(file, dst_folder)
            print(f"✅ 복사됨: {os.path.basename(file)} / Copied: {os.path.basename(file)}")
        except Exception as e:
            print(f"❌ 복사 실패: {str(e)} / Copy failed: {str(e)}")

# 개인 경로에 결과 복사 / Copy to your own path
drive_root = f"/content/gdrive/MyDrive/{MY_PATH}"
safe_copy(f'{local_done_vis}/*', f'{drive_root}/done_vis/')
safe_copy(f'{local_done_csv}/*', f'{drive_root}/done_csv/')
safe_copy(f'{local_alljt_vis}/*', f'{drive_root}/ALL_JT_VIS/')
safe_copy(f'{local_alljt_csv}/*', f'{drive_root}/ALL_JT_CSV/')
safe_copy(f'{local_vis_dir}/*', f'{drive_root}/vis/')
safe_copy(f'{local_json_dir}/*', f'{drive_root}/json/')

Mounted at /content/gdrive
🔍 패키지 버전 확인 중... / Checking package versions...
✅ torch: 2.0.1+cu118
❌ torch는 2.0.1 필요, 현재 2.0.1+cu118 / torch requires 2.0.1, but found 2.0.1+cu118
✅ torchvision: 0.15.2+cu118
❌ torchvision는 0.15.2 필요, 현재 0.15.2+cu118 / torchvision requires 0.15.2, but found 0.15.2+cu118
✅ torchaudio: 2.0.2+cu118
❌ torchaudio는 2.0.2 필요, 현재 2.0.2+cu118 / torchaudio requires 2.0.2, but found 2.0.2+cu118
✅ mmcv: 2.0.1
✅ mmdet: 3.2.0
❌ mmpose 미설치 / mmpose not installed
✅ mmengine: 0.10.7

🚀 필수 패키지 설치 중... / Installing required packages...
Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached transformers-4.30.2-py3-none-any.whl.metadata (113 kB)
Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.15.2 requires accelerate>=0.21.0, which is not installed.
sentence-transformer

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.7' (you have '2.0.6'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/body_2d_keypoint/topdown_heatmap/coco/td-hm_hrnet-w32_8xb64-210e_coco-256x192-81c58e40_20220909.pth
05/20 22:50:09 - mmengine - WARNING - Failed to search registry with scope "mmpose" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpose" is a correct scope, or whether the registry is initialized.
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth


Downloading: "https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth" to /root/.cache/torch/hub/checkpoints/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth


05/20 22:50:16 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.
처리 중: sport3.mp4 / Processing: sport3.mp4


/usr/local/lib/python3.11/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


05/20 22:58:23 - mmengine - INFO - the output video has been saved at /content/work/vis/sport3.mp4
✅ 처리 완료: sport3.mp4 / Processed: sport3.mp4
처리 중: gait4.mp4 / Processing: gait4.mp4
05/20 23:13:53 - mmengine - INFO - the output video has been saved at /content/work/vis/gait4.mp4
✅ 처리 완료: gait4.mp4 / Processed: gait4.mp4
처리 중: gait2.mp4 / Processing: gait2.mp4
05/20 23:44:16 - mmengine - INFO - the output video has been saved at /content/work/vis/gait2.mp4
✅ 처리 완료: gait2.mp4 / Processed: gait2.mp4
처리 중: gait3.mp4 / Processing: gait3.mp4
05/21 00:17:20 - mmengine - INFO - the output video has been saved at /content/work/vis/gait3.mp4
✅ 처리 완료: gait3.mp4 / Processed: gait3.mp4
처리 중: sport1.mp4 / Processing: sport1.mp4
05/21 00:31:42 - mmengine - INFO - the output video has been saved at /content/work/vis/sport1.mp4
✅ 처리 완료: sport1.mp4 / Processed: sport1.mp4
처리 중: sport4.mp4 / Processing: sport4.mp4
05/21 01:02:02 - mmengine - INFO - the output video has been saved at /content/work/vis/sp